In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [35]:
import lightgbm as lgb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

import itertools
from scipy import stats

In [3]:
import sys
sys.path.append('../')

In [4]:
from language_source_classifer.features.build_features import Vectorizer
from language_source_classifer.data.dataframes import Data

In [5]:
data = Data()

In [6]:
train_df = data.train_df
test_df = data.test_df

In [26]:
X_train = train_df['code']
X_test = test_df['code']

In [27]:
y_train = train_df['language']
y_test = test_df['language']

In [28]:
tokenizer_ls = ['char', 'alphanum']
n_gram_ls = [(1,1), (2,2), (3,3), (1,2), (1,3)]

In [29]:
preds = []
for tokenizer, n_gram in itertools.product(tokenizer_ls, ngrams_ls):
    vec = Vectorizer(tokenizer=tokenizer, n_gram=n_gram)
    X_train_vec = vec.vectorize_df(X_train)
    X_test_vec = vec.vectorize_df(X_test)
    model = lgb.LGBMClassifier()
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    acc_score = accuracy_score(y_test, y_pred)
    print(f'{tokenizer}, {n_gram}, acc: {acc_score}')
    preds.append(y_pred)

/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


char, (1, 1), acc: 0.8725366876310272


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


char, (2, 2), acc: 0.940041928721174


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


alphanum, (3, 3), acc: 0.7882599580712788


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


alphanum, (1, 2), acc: 0.929559748427673
alphanum, (1, 3), acc: 0.9174004192872117


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [32]:
p1, p2, p3 = preds[1], preds[5], preds[9]

In [38]:
accuracy_score(p2, p3)

0.9362683438155136

In [42]:
pmode = stats.mode([p1, p2, p3]).mode.flatten()

In [43]:
accuracy_score(y_test, pmode)

0.9421383647798742

In [44]:
preds = []
for tokenizer, n_gram in itertools.product(tokenizer_ls, ngrams_ls):
    vec = Vectorizer(n_features=2**12, tokenizer=tokenizer, n_gram=n_gram)
    X_train_vec = vec.vectorize_df(X_train)
    X_test_vec = vec.vectorize_df(X_test)
    model = lgb.LGBMClassifier()
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    acc_score = accuracy_score(y_test, y_pred)
    print(f'{tokenizer}, {n_gram}, acc: {acc_score}')
    preds.append(y_pred)

/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


char, (1, 1), acc: 0.8721174004192872


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


char, (2, 2), acc: 0.9467505241090147


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


alphanum, (3, 3), acc: 0.8515723270440252


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


alphanum, (1, 2), acc: 0.9454926624737946
alphanum, (1, 3), acc: 0.939203354297694


/home/klui/.local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
